# Latent Dirichlet Allocation (LDA)

Sklearn example from https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### Load data from the 20 news groups data

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print(documents[0])

Well i'm not sure about the story nad it did seem biased. What
I disagree with is your statement that the U.S. Media is out to
ruin Israels reputation. That is rediculous. The U.S. media is
the most pro-israeli media in the world. Having lived in Europe
I realize that incidences such as the one described in the
letter have occured. The U.S. media as a whole seem to try to
ignore them. The U.S. is subsidizing Israels existance and the
Europeans are not (at least not to the same degree). So I think
that might be a reason they report more clearly on the
atrocities.
	What is a shame is that in Austria, daily reports of
the inhuman acts commited by Israeli soldiers and the blessing
received from the Government makes some of the Holocaust guilt
go away. After all, look how the Jews are treating other races
when they got power. It is unfortunate.



### Prepare the data for LDA

In [3]:
num_features = 1000

# LDA can only uses raw term counts
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=num_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)

In [4]:
tf_feature_names = tf_vectorizer.get_feature_names() #theses are the words in our bag of words

### Build the LDA model

In [5]:
num_topics = 10

# Run LDA
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=5, learning_method='online',random_state=0, n_jobs=-1)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [6]:
lda.components_

array([[1.00224371e-01, 2.94490425e+02, 1.00009572e-01, ...,
        4.58066266e+00, 1.40763399e+02, 1.08318812e+02],
       [1.00335073e-01, 1.99507980e+01, 1.00066313e-01, ...,
        2.71899073e+02, 4.59168261e+00, 1.00093416e-01],
       [2.40288744e+00, 1.00044968e-01, 2.06517163e-01, ...,
        1.01934120e-01, 1.00037526e-01, 1.00024359e-01],
       ...,
       [1.11080956e-01, 1.00025849e-01, 1.00576955e-01, ...,
        1.00000721e-01, 1.00258149e-01, 1.00005005e-01],
       [1.00300329e-01, 1.00184041e-01, 1.00014551e-01, ...,
        2.58994253e+02, 4.49397641e+01, 1.54072534e+02],
       [1.05807440e+03, 3.57307803e+02, 2.19107405e+02, ...,
        1.67731290e-01, 1.03143091e+02, 3.38903202e+01]])

### Display the top ten words for each topic

In [8]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-num_top_words - 1:-1]]))

num_top_words = 10
display_topics(lda, tf_feature_names, num_top_words)

Topic 0:
people government gun armenian said armenians states war israel state
Topic 1:
people don like right point make know think use question
Topic 2:
space program db output data entry nasa use build line
Topic 3:
key encryption chip government use keys public clipper security law
Topic 4:
edu file com available mail information ftp files list send
Topic 5:
god people jesus does believe say think christian bible true
Topic 6:
windows use drive thanks does card problem like know using
Topic 7:
ax max b8f g9v a86 pl 145 1d9 0t 34u
Topic 8:
just don like think know good time didn going ve
Topic 9:
10 00 20 15 25 12 14 11 17 16


### Model evaluation

Model [perplexity](https://en.wikipedia.org/wiki/Perplexity) is often used in LDA to evaluate how well a model predicts a sample.

In [9]:
print("Model perplexity: {0:0.3f}".format(lda.perplexity(tf)))

Model perplexity: 265.308


## Show how to do LDA in gensim

Example from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

If you don't already have gensim installed:  
`$ pip install -U gensim`  

And pyLDAVis:  
`$ pip install pyldavis`

### Imports

In [12]:
import pandas as pd
import numpy as np

#gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package wordnet to /home/frank/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load data

In [13]:
data = pd.read_csv('data/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

### Preprocess data

In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [15]:
stemmer = SnowballStemmer('english')
processed_docs = documents['headline_text'].map(preprocess)

In [16]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words

In [17]:
#create dictionary
id2word = gensim.corpora.Dictionary(processed_docs)

#create corpus
texts = processed_docs

#Term Document Frequency
bow_corpus = [id2word.doc2bow(text) for text in texts]

In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in bow_corpus]

[[('broadcast', 1), ('communiti', 1), ('decid', 1), ('licenc', 1)],
 [('awar', 1), ('defam', 1), ('wit', 1)],
 [('call', 1), ('infrastructur', 1), ('protect', 1), ('summit', 1)],
 [('aust', 1), ('rise', 1), ('staff', 1), ('strike', 1)],
 [('strike', 1), ('affect', 1), ('australian', 1), ('travel', 1)],
 [('ambiti', 1), ('jump', 1), ('olsson', 1), ('tripl', 1), ('win', 1)],
 [('antic', 1), ('barca', 1), ('break', 1), ('delight', 1), ('record', 1)],
 [('aussi', 1),
  ('match', 1),
  ('memphi', 1),
  ('qualifi', 1),
  ('stosur', 1),
  ('wast', 1)],
 [('aust', 1), ('address', 1), ('council', 1), ('iraq', 1), ('secur', 1)],
 [('australia', 1), ('lock', 1), ('timet', 1)],
 [('iraq', 1), ('australia', 1), ('contribut', 1), ('million', 1)],
 [('barca', 1), ('record', 1), ('birthday', 1), ('celebr', 1), ('robson', 1)],
 [('ahead', 1), ('bathhous', 1), ('plan', 1)],
 [('championship', 1), ('cycl', 1), ('hop', 1), ('launceston', 1)],
 [('plan', 1), ('boost', 1), ('paroo', 1), ('suppli', 1), ('wat

In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=id2word, passes=2, workers=2)

### View topics in the LDA model

In [19]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.030*"elect" + 0.022*"warn" + 0.019*"live" + 0.017*"drug" + 0.017*"donald" '
  '+ 0.011*"campaign" + 0.011*"violenc" + 0.010*"john" + 0.010*"young" + '
  '0.010*"show"'),
 (1,
  '0.032*"govern" + 0.024*"say" + 0.019*"help" + 0.015*"state" + '
  '0.015*"report" + 0.014*"labor" + 0.013*"servic" + 0.013*"peopl" + '
  '0.012*"feder" + 0.012*"victoria"'),
 (2,
  '0.026*"polic" + 0.025*"queensland" + 0.023*"attack" + 0.020*"kill" + '
  '0.019*"crash" + 0.018*"melbourn" + 0.016*"die" + 0.016*"interview" + '
  '0.016*"brisban" + 0.015*"shoot"'),
 (3,
  '0.018*"rural" + 0.015*"death" + 0.015*"power" + 0.014*"call" + '
  '0.013*"investig" + 0.013*"farmer" + 0.013*"communiti" + 0.013*"concern" + '
  '0.011*"farm" + 0.010*"royal"'),
 (4,
  '0.033*"trump" + 0.023*"adelaid" + 0.020*"perth" + 0.017*"final" + '
  '0.017*"world" + 0.015*"open" + 0.015*"win" + 0.013*"test" + 0.012*"lose" + '
  '0.011*"flood"'),
 (5,
  '0.034*"charg" + 0.032*"court" + 0.025*"murder" + 0.021*"face" + '
  '0.018*"

### Compute Model Perplexity and Coherence Score (interpretability of the model)



In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.242384565660066

Coherence Score:  0.2074296670593884


### Visualize the topics-keywords

In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

/home/frank/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.164614 -0.153302       1        1  12.043525
2      0.066016  0.239264       2        1  10.887059
3     -0.222016  0.019840       3        1  10.520773
1     -0.204291 -0.154468       4        1  10.496809
7      0.156606 -0.140525       5        1  10.246866
5      0.019492  0.287377       6        1   9.893742
4      0.260633 -0.049274       7        1   9.785935
9      0.003469  0.018832       8        1   8.792999
0     -0.099295  0.093665       9        1   8.785832
6      0.184001 -0.161409      10        1   8.546458, topic_info=     Category          Freq        Term         Total  loglift  logprob
38    Default  26742.000000   australia  26742.000000  30.0000  30.0000
326   Default  16903.000000       charg  16903.000000  29.0000  29.0000
2543  Default  16871.000000      govern  16871.000000  28.0000  28.0000
7880  Default  16359.000000       trump  16359.000000  27.0000  27.0000
217   Default  15797.000000       court  15797.000000  26.0000  26.0000
784   Default  13360.000000       elect  13360.000000  25.0000  25.0000
48    Default  14489.000000        plan  14489.000000  24.0000  24.0000
2104  Default  13718.000000  queensland  13718.000000  23.0000  23.0000
81    Default  12855.000000      attack  12855.000000  22.0000  22.0000
307   Default  12225.000000      murder  12225.000000  21.0000  21.0000
193   Default  12902.000000      nation  12902.000000  20.0000  20.0000
1415  Default  11557.000000       south  11557.000000  19.0000  19.0000
1227  Default  10601.000000    canberra  10601.000000  18.0000  18.0000
998   Default  11188.000000     adelaid  11188.000000  17.0000  17.0000
241   Default  28170.000000       polic  28170.000000  16.0000  16.0000
425   Default   9964.000000     countri   9964.000000  15.0000  15.0000
541   Default  11035.000000        kill  11035.000000  14.0000  14.0000
1070  Default  11555.000000      school  11555.000000  13.0000  13.0000
433   Default  10292.000000        face  10292.000000  12.0000  12.0000
16    Default  21912.000000  australian  21912.000000  11.0000  11.0000
351   Default   9744.000000        warn   9744.000000  10.0000  10.0000
35    Default  11169.000000     council  11169.000000   9.0000   9.0000
1860  Default   9803.000000       perth   9803.000000   8.0000   8.0000
198   Default  10263.000000       crash  10263.000000   7.0000   7.0000
227   Default   9833.000000       coast   9833.000000   6.0000   6.0000
116   Default   9803.000000        help   9803.000000   5.0000   5.0000
146   Default  13662.000000       death  13662.000000   4.0000   4.0000
873   Default  12843.000000    melbourn  12843.000000   3.0000   3.0000
1142  Default  17074.000000        year  17074.000000   2.0000   2.0000
1136  Default   9447.000000      market   9447.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
2298  Topic10   3714.655029     michael   3715.485596   2.4594  -4.7536
2698  Topic10   3589.719971      chines   3590.550293   2.4594  -4.7878
1309  Topic10   3321.172363       great   3322.002930   2.4594  -4.8655
530   Topic10   3072.466309       stand   3073.296631   2.4594  -4.9434
290   Topic10   3067.560547       white   3068.391113   2.4594  -4.9450
917   Topic10   2926.252686       point   2927.083008   2.4594  -4.9921
5470  Topic10   2848.665527       mount   2849.496094   2.4594  -5.0190
6368  Topic10   2843.697998         jam   2844.528320   2.4594  -5.0208
2802  Topic10   2535.111328       scott   2535.941650   2.4593  -5.1356
1159  Topic10   2482.955078      export   2483.785645   2.4593  -5.1564
1221  Topic10   2467.098877       owner   2467.929443   2.4593  -5.1628
557   Topic10   2326.927490         fan   2327.758057   2.4593  -5.2213
105   Topic10   2165.088867       donat   2165.919434   2.4593  -5.2934
1206  Topic10   2140.023682     product   2140.854004   2.4593  -5.30